In [1]:
print('Hello, World!')

Hello, World!


In [2]:
import numpy as np

In [5]:
# Sample deep learning network using PyTorch to test environment
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import pandas as pd
import logging
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f'Using device: {device}')
# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 10
# Data preparation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
# Model, loss function, optimizer, scheduler
model = resnet18(pretrained=False, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
# TensorBoard setup
log_dir = './logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
writer = SummaryWriter(log_dir=log_dir)
# Training and evaluation functions
def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    logger.info(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    writer.add_scalar('Train/Loss', epoch_loss, epoch)
    writer.add_scalar('Train/Accuracy', epoch_acc, epoch)

def evaluate(epoch):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f'Evaluating Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    logger.info(f'Validation Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    writer.add_scalar('Val/Loss', epoch_loss, epoch)
    writer.add_scalar('Val/Accuracy', epoch_acc, epoch)
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_df = pd.DataFrame(cm, index=train_dataset.classes, columns=train_dataset.classes)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title(f'Confusion Matrix Epoch {epoch+1}')
    cm_path = os.path.join(log_dir, f'confusion_matrix_epoch_{epoch+1}.png')
    plt.savefig(cm_path)
    plt.close()
    writer.add_image('Confusion_Matrix', plt.imread(cm_path), epoch, dataformats='HWC')
# Main training loop
start_time = time.time()
for epoch in range(num_epochs):
    train_one_epoch(epoch)
    evaluate(epoch)
    scheduler.step()
end_time = time.time()
logger.info(f'Training complete in {(end_time - start_time)/60:.2f} minutes')
# Save the model
model_path = './resnet18_cifar10.pth'
torch.save(model.state_dict(), model_path)
# Close the TensorBoard writer
writer.close()


INFO:__main__:Using device: cpu
100%|██████████| 170M/170M [04:52<00:00, 583kB/s]  
Training Epoch 1/10: 100%|██████████| 391/391 [03:08<00:00,  2.08it/s]
INFO:__main__:Epoch 1, Loss: 1.5667, Accuracy: 0.4309
Evaluating Epoch 1/10: 100%|██████████| 79/79 [00:31<00:00,  2.48it/s]
INFO:__main__:Validation Loss: 1.3406, Accuracy: 0.5161
Training Epoch 2/10: 100%|██████████| 391/391 [03:10<00:00,  2.06it/s]
INFO:__main__:Epoch 2, Loss: 1.1887, Accuracy: 0.5756
Evaluating Epoch 2/10: 100%|██████████| 79/79 [00:31<00:00,  2.52it/s]
INFO:__main__:Validation Loss: 1.1340, Accuracy: 0.6062
Training Epoch 3/10: 100%|██████████| 391/391 [03:06<00:00,  2.09it/s]
INFO:__main__:Epoch 3, Loss: 1.0244, Accuracy: 0.6370
Evaluating Epoch 3/10: 100%|██████████| 79/79 [00:30<00:00,  2.55it/s]
INFO:__main__:Validation Loss: 0.9758, Accuracy: 0.6561
Training Epoch 4/10: 100%|██████████| 391/391 [03:05<00:00,  2.11it/s]
INFO:__main__:Epoch 4, Loss: 0.9329, Accuracy: 0.6703
Evaluating Epoch 4/10: 100%|███████